## Imports

In [68]:
!pip -q install openai
!pip -q install loguru

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [74]:
import os
import random
import requests
import psycopg2
import openai
from sqlalchemy import create_engine
from loguru import logger

## Configs
Environment variables can be used by integration in deepnote very easy and pragmatic

In [8]:
openai.api_key = os.environ["OPENAI_API_KEY"]
db_url_string = os.environ["DB_URL"]

In [61]:
engine = create_engine(os.getenv("DB_URL"), isolation_level="SERIALIZABLE", echo=False)

In [56]:
# Prevent echoing 
import logging
logging.basicConfig()
logging.getLogger('sqlalchemy').setLevel(logging.ERROR)

### Create a db table in deepnote style

In [ ]:


df_1 = _deepnote_execute_sql("""CREATE TABLE  if not exists my_chats
( 
 dt timestamp,
 question text,
 answer text
)
""", 'SQL_E296F575_C606_49CB_9261_D2C5618B5B0D')
df_1

### Chat 

In [76]:
def asking(prompt="Ask me something smart"):
    response = openai.Completion.create(
    model="text-davinci-003", 
    prompt=prompt, 
    temperature=0.7, 
    max_tokens=100    
    )["choices"][0]["text"].strip()
    return response

In [78]:
# Let's have a talk with itself
# Init question
response_init = asking()
response_init

'What is the history of the scientific method?'

In [79]:
stupid_answers = ["For real", "This can't be true", "Please be more concrete", "I don't think so"]
for i in range(5):
    response = asking(prompt=response_init)
    #logger.info(f"[*] Question: {response_init}")
    #logger.info(f"[*] Response: {response}")
    engine.execute("""INSERT INTO my_chats (dt, question, answer)  
    VALUES (Now(), %s, %s )""", response_init, response)
    response_init = random.choice(stupid_answers)

### Show discussion

In [80]:


df_2 = _deepnote_execute_sql("""Select  * from my_chats;
--Delete from my_chats;
""", 'SQL_E296F575_C606_49CB_9261_D2C5618B5B0D')
df_2

,dt,question,answer
0,2023-02-15 23:52:00.051023,What is the history of the scientific method?,"The scientific method has evolved over time, b..."
1,2023-02-15 23:52:01.207042,This can't be true,No statement is ever true or false without con...
2,2023-02-15 23:52:05.244384,This can't be true,"!\n\nWe understand why you may have doubts, bu..."
3,2023-02-15 23:52:10.746006,For real,"estate agents, the ideal way to market themsel..."
4,2023-02-15 23:52:16.301534,I don't think so,".\n\nNo, there is no scientific evidence to su..."


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a38f32e0-6b3e-4770-a24a-9a021096739d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>